In [1]:
import os
import csv

samples = []
with open('data2/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
    samples.remove(samples[0])

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print(samples[0])
print('this is the first line in the training samples')

import cv2
import numpy as np
import sklearn
import keras
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

mypath = 'C:\\Users\\Marwan\\Downloads\\CarND-Behavioral-Cloning-P3-master\\data2\\IMG\\center_2018_02_06_18_38_05_682.jpg'
mypath = mypath[61:]
mypath = mypath.replace('\\','/')
myimage = cv2.imread(mypath)
print(myimage.shape)
print('this is the shape of a single image')


def mygenerator(samples, batch_size=32,counter=3):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(1, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = batch_sample[0]
                name = name[61:]
                name = name.replace('\\','/')
                
                name_left = batch_sample[1]
                name_left = name_left[61:]
                name_left = name_left.replace('\\','/')
                
                name_right = batch_sample[2]
                name_right = name_right[61:]
                name_right = name_right.replace('\\','/')
                
                center_image = cv2.imread(name)
                left_image = cv2.imread(name_left)
                right_image = cv2.imread(name_right)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                images.append(left_image)
                angles.append(center_angle)
                images.append(right_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = mygenerator(train_samples, batch_size=96,counter=3)
validation_generator = mygenerator(validation_samples, batch_size=96,counter=3)


model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Cropping2D(cropping=((50,20), (0,0)),data_format='channels_last', input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/127.5 - 1.))
model.add(Conv2D(12,(5,5),strides=(2, 2), padding='valid',activation='relu'))
model.add(Conv2D(24,(5,5),strides=(2, 2), padding='valid',activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=2, verbose = 1)

"""
If the above code throw exceptions, try 
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=5, verbose = 1)
"""

['C:\\Users\\Marwan\\Downloads\\CarND-Behavioral-Cloning-P3-master\\data2\\IMG\\center_2018_02_06_18_38_05_682.jpg', 'C:\\Users\\Marwan\\Downloads\\CarND-Behavioral-Cloning-P3-master\\data2\\IMG\\left_2018_02_06_18_38_05_682.jpg', 'C:\\Users\\Marwan\\Downloads\\CarND-Behavioral-Cloning-P3-master\\data2\\IMG\\right_2018_02_06_18_38_05_682.jpg', '0', '0', '0', '17.73556']
this is the first line in the training samples


Using TensorFlow backend.


(160, 320, 3)
this is the shape of a single image
Epoch 1/2
  17/1060 [..............................] - ETA: 2:49:03 - loss: 43.6436

KeyboardInterrupt: 

In [9]:
model.save('my_model2.h5')